<a href="https://colab.research.google.com/github/quotation3/TIL/blob/master/%ED%95%9C%EA%B5%AD%EC%9D%80%ED%96%89%EB%85%BC%EB%AC%B8%EA%B5%AC%ED%98%84_%EC%BD%9C%EA%B8%88%EB%A6%AC%ED%81%AC%EB%A1%A4%EB%A7%81_%EB%9D%BC%EB%B2%A8%EB%A7%81%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

In [52]:
#네이버금융에서 콜금리 크롤링
dates = []
rates = []

for j in range(1,556):
    res = requests.get("https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}".format(j))
    bs = BeautifulSoup(res.text, "html.parser")
    # print(bs.text)
    date = bs.select('td.date')
    rate = bs.select('td.num')

    for i in range(len(date)):
        d=datetime.strptime((date[i].text.replace('\t','').replace('\n','')), '%Y.%m.%d').strftime('%Y-%m-%d')
        dates.append(d)
        rates.append(rate[3*i].text)

In [53]:
data = np.array([dates,rates])
df_rate = pd.DataFrame(data.T, columns=['date','call_rate'])

In [54]:
# 중간중간 빈날짜를 채워줘야함.
from datetime import datetime, timedelta
# start_date부터 end_date까지 날짜 리스트 생성
start_date = datetime.strptime('20041201', '%Y%m%d')
end_date = datetime.strptime('20200722', '%Y%m%d')

str_date_list = []
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): # start_date가 end_date랑 같아지기 전까지 1씩 늘어나면서 리스트에 추가
    str_date_list.append(start_date.strftime('%Y-%m-%d'))
    start_date += timedelta(days=1)
str_date_list.append(end_date.strftime('%Y-%m-%d')) # end_date 포함안됐으므로 따로 추가
    
date_list=np.array([str_date_list])
all_date  = pd.DataFrame(date_list.T, columns=['date'])
# print(all_date)

In [55]:
all_date.head(20)

,date
0,2004-12-01
1,2004-12-02
2,2004-12-03
3,2004-12-04
4,2004-12-05
5,2004-12-06
6,2004-12-07
7,2004-12-08
8,2004-12-09
9,2004-12-10


In [56]:
df = pd.merge(df_rate,all_date, on='date', how='outer') # 기존크롤링데이터와 전체날짜를 date기준으로 outer 조인
df_callrate = df.sort_values(by='date' ,ascending=False).fillna(method='bfill') # 내림차순으로 정렬후, 결측치는 가장 최근 과거 값으로 대입
df_callrate

,date,call_rate
0,2020-07-27,0.49
1,2020-07-24,0.48
2,2020-07-23,0.46
3,2020-07-22,0.45
4,2020-07-21,0.46
...,...,...
3886,2004-12-05,3.26
3885,2004-12-04,3.26
3882,2004-12-03,3.26
3883,2004-12-02,3.26


In [ ]:
df_callrate.to_csv('call_rate.csv', index=False, encoding="utf-8")

In [57]:
# 'rate_before30' 이란 열을 만들고 call_rate와 똑같은 값을 복사
df_callrate['rate_before30']=df_callrate['call_rate']
df_callrate['rate_gap']=''

In [58]:
df_callrate

,date,call_rate,rate_before30,rate_gap
0,2020-07-27,0.49,0.49,
1,2020-07-24,0.48,0.48,
2,2020-07-23,0.46,0.46,
3,2020-07-22,0.45,0.45,
4,2020-07-21,0.46,0.46,
...,...,...,...,...
3886,2004-12-05,3.26,3.26,
3885,2004-12-04,3.26,3.26,
3882,2004-12-03,3.26,3.26,
3883,2004-12-02,3.26,3.26,


In [59]:
# 30일 전의 금리를 확인하기 위해 'rate_before30'의 행을 30행 위으로 내림
df_callrate.rate_before30 = df_callrate.rate_before30.shift(-30)
df_callrate

,date,call_rate,rate_before30,rate_gap
0,2020-07-27,0.49,0.47,
1,2020-07-24,0.48,0.48,
2,2020-07-23,0.46,0.47,
3,2020-07-22,0.45,0.47,
4,2020-07-21,0.46,0.46,
...,...,...,...,...
3886,2004-12-05,3.26,NaN,
3885,2004-12-04,3.26,NaN,
3882,2004-12-03,3.26,NaN,
3883,2004-12-02,3.26,NaN,


In [60]:
# 상승하락 여부를 라벨링하기위한 빈 열 생성
df_callrate['label']=''

In [61]:
# 'call_rate'에서 'rate_after30'를 뺏을때 0.03이상이면  하락(dovish), -0.03이하면 상승(hawkish), 그 사이면 변화없음(no_change) 라벨링
for i in range(len(df_callrate)):
    df_callrate['rate_gap'][i] = float(df_callrate['call_rate'][i])-float(df_callrate['rate_before30'][i])
    if float(df_callrate['call_rate'][i])-float(df_callrate['rate_before30'][i]) > 0.03:
        df_callrate['label'][i] = df_callrate['label'][i].replace('', 'hawkish')

    elif float(df_callrate['call_rate'][i])-float(df_callrate['rate_before30'][i]) < -0.03:
        df_callrate['label'][i] = df_callrate['label'][i].replace('', 'dovish')
    else:
        df_callrate['label'][i] = df_callrate['label'][i].replace('', 'no_change')

In [62]:
df_callrate

,date,call_rate,rate_before30,rate_gap,label
0,2020-07-27,0.49,0.47,0.02,no_change
1,2020-07-24,0.48,0.48,0,no_change
2,2020-07-23,0.46,0.47,-0.01,no_change
3,2020-07-22,0.45,0.47,-0.02,no_change
4,2020-07-21,0.46,0.46,0,no_change
...,...,...,...,...,...
3886,2004-12-05,3.26,NaN,NaN,no_change
3885,2004-12-04,3.26,NaN,NaN,no_change
3882,2004-12-03,3.26,NaN,NaN,no_change
3883,2004-12-02,3.26,NaN,NaN,no_change


In [63]:
df_callrate.to_csv('label_call_rate.csv', index=False, encoding="utf-8")